# Model Building
Where I build the CNN model

In [13]:
import sys
import os

from dotenv import load_dotenv

import torch
import torch.nn as nn
import torch.nn.functional as F

# add parent directory to path so I can import from data)
sys.path.append(os.getenv('HOME_DIR'))

from data.import_data import load_dataset

load_dotenv()

True

In [14]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
loss_func = nn.BCELoss()

In [15]:
class Model(nn.Module):
    '''
    A CNN with 1 convolutional layer and 1 linear layer.
    '''
    def __init__(self, num_channels):
        super().__init__()
                
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=1, 
                      out_channels=num_channels, 
                      kernel_size=(3, 3),
                      stride=1,),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2), stride=2, padding=0),
        )
        self.layer2 = nn.Linear(num_channels * 63 * 39, 2)

        
    def forward(self, x):
        x = self.layer1(x)
        x = torch.flatten(x, 1)
        x = F.softmax(self.layer2(x), dim=1)
        return x

In [16]:
model = Model(num_channels=16).to(device)

In [17]:
# view model parameters
for name, param in model.named_parameters():
    print(f"Name: {name}")
    print(f"Size: {param.size()}")

Name: layer1.0.weight
Size: torch.Size([16, 1, 3, 3])
Name: layer1.0.bias
Size: torch.Size([16])
Name: layer2.weight
Size: torch.Size([2, 39312])
Name: layer2.bias
Size: torch.Size([2])


In [18]:
train_dl = load_dataset(os.path.join(os.getenv('HOME_DIR'),'data/train_meta.csv'), files_to_load=100)

In [19]:
for batch, labels in train_dl:
    batch = batch.to(device)
    labels = labels.to(device)
    output = model(batch)
    loss = loss_func(output, labels)
    #print(f"Loss: {loss}")
    print(output)
    print(labels)

    break

tensor([[0.0757, 0.9243],
        [0.3537, 0.6463],
        [0.4878, 0.5122],
        [0.4318, 0.5682],
        [0.4652, 0.5348],
        [0.4648, 0.5352],
        [0.0853, 0.9147],
        [0.4224, 0.5776],
        [0.4603, 0.5397],
        [0.4224, 0.5776],
        [0.6064, 0.3936],
        [0.4636, 0.5364],
        [0.2194, 0.7806],
        [0.5009, 0.4991],
        [0.4654, 0.5346],
        [0.9919, 0.0081],
        [0.4685, 0.5315],
        [0.8559, 0.1441],
        [0.9665, 0.0335],
        [0.4004, 0.5996],
        [0.2156, 0.7844],
        [0.4661, 0.5339],
        [0.7323, 0.2677],
        [0.4723, 0.5277],
        [0.9356, 0.0644],
        [0.5405, 0.4595],
        [0.0340, 0.9660],
        [0.5571, 0.4429],
        [0.4161, 0.5839],
        [0.3593, 0.6407],
        [0.9343, 0.0657],
        [0.3876, 0.6124]], grad_fn=<SoftmaxBackward0>)
tensor([[0., 1.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [0., 1.],
        [0., 1.],
        [1., 0.],
        [1., 

In [20]:
model

Model(
  (layer1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Linear(in_features=39312, out_features=2, bias=True)
)